In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ppxf_functions_Z import (
    load_spectrum,
    process_spectrum,
    rebin_to_log,
    make_noise,
    calculate_velscale_fwhm,
    run_ppxf,
    plot_ppxf,
)

In [2]:
# --- Inputs ---
galaxy_files = [
    (f"../calibrated data/WB{i}_data_orth.npy", f"../calibrated data/WB{i}_waveobs_orth.npy")
    for i in [1, 2, 5, 6, 7]
]

# Match each dataset to its redshift (same order)
redshifts = [
    0.017206345, 
    0.038136434,
    0.041185744,
    0.045207640,
    0.042784720,
    0.017624998,
    0.046458002
]


In [3]:
window_size = 10
step        = 1
scale       = 0.4   # arcsec per pixel
pix_min, pix_max = 150, 250

all_results = {}

# --- Loop over galaxies ---
for (data_file, wave_file), z in zip(galaxy_files, redshifts):

    print(f"\nRunning galaxy {data_file} with z={z}")

    raw = np.load(data_file)       # flux × spatial pixel
    waveem = np.load(wave_file)

    results = []
    raw_roi = raw[:, pix_min:pix_max]

    for start in range(0, raw_roi.shape[1] - window_size + 1, step):
        end = start + window_size
        spectrum = raw_roi[:, start:end].sum(axis=1)

        # Build DataFrame
        df_spec = pd.DataFrame({
            "waveem": waveem,
            "flux":   spectrum
        })

        # Process, rebin, noise
        df_cut = process_spectrum(df_spec)
        df_rb, lam, flux_rb, ln_w = rebin_to_log(df_cut)

        noise = make_noise(df_rb)
        velscale, fwhm = calculate_velscale_fwhm(ln_w, lam)
        center = start + window_size//2 + pix_min

        # Run pPXF with galaxy's redshift
        pp = run_ppxf(lam, fwhm, velscale, df_rb, noise, redshift=z)

        print(f'  PIXEL: {center}')

        # Store result
        results.append((center, df_rb, pp))

    # Save per galaxy
    all_results[data_file] = results


Running galaxy ../calibrated data/WB1_data_orth.npy with z=0.017206345
Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      5114       120
 comp.  1:      5114        50
chi2/DOF: 0.9691; DOF: 1599; degree = 4; mdegree = 10
method = capfit; Jac calls: 1; Func calls: 17; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha     0.1032       5.8    5114    50
Comp:  1             [SII]6716     0.9719       5.8    5114    50
Comp:  1             [SII]6731      0.000       5.8    5114    50
Comp:  1           [NII]6583_d     0.3090       7.3    5114    50
---------------

 Best Fit:       Vel     sigma
 comp.  0:      4594        72
 comp.  1:      5231        63
chi2/DOF: 0.9698; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 158; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000       1.5    5231    63
Comp:  1             [SII]6716      1.158       1.5    5231    63
Comp:  1             [SII]6731      0.000       1.5    5231    63
Comp:  1           [NII]6583_d      0.000       1.9    5231    63
-----------------------------------------------------------------
  PIXEL: 163
Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best 

/Users/owner1/anaconda3/lib/python3.11/site-packages/capfit/capfit.py:799: RuntimeWarning: invalid value encountered in scalar divide
  ratio = actred/prered


Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4659       607
 comp.  1:      5207        73
chi2/DOF: 0.9206; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 14; Func calls: 215; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000      0.67    5207    73
Comp:  1             [SII]6716      0.000      0.67    5207    73
Comp:  1             [SII]6731      0.000      0.67    5207    73
Comp:  1           [NII]6583_d      0.000      0.84    5207    73
-----------------------------------------------------------------
  PIXEL: 172
Emissi

Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4972        56
 comp.  1:      5049         1
chi2/DOF: 0.9289; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 170; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 1/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000      0.62    5049     1
Comp:  1             [SII]6716      0.000      0.62    5049     1
Comp:  1             [SII]6731      0.000      0.62    5049     1
Comp:  1           [NII]6583_d      0.000      0.79    5049     1
-----------------------------------------------------------------
  PIXEL: 181
Emissi

...

Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     11851       491
 comp.  1:     11852       174
chi2/DOF: 0.3133; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 122; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      25.37       2.2   11852   174
Comp:  1           [NII]6583_d      11.96       2.8   11852   174
-----------------------------------------------------------------
  PIXEL: 202
Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OI

Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     12048         1
 comp.  1:     12707         1
chi2/DOF: 0.5375; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 13; Func calls: 201; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      7.968       6.8   12707     1
Comp:  1           [NII]6583_d      229.1       8.6   12707     1
-----------------------------------------------------------------
  PIXEL: 212
Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[O

Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     12355         1
 comp.  1:     12388       238
chi2/DOF: 0.9518; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 129; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      32.58       11.   12388   238
Comp:  1           [NII]6583_d      27.91       14.   12388   238
-----------------------------------------------------------------
  PIXEL: 222
Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OI

Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     12201       454
 comp.  1:     12516         1
chi2/DOF: 0.9715; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 114; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 2/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      3.222       24.   12516     1
Comp:  1           [NII]6583_d      0.000       30.   12516     1
-----------------------------------------------------------------
  PIXEL: 232
Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OI

Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     12312        24
 comp.  1:     12561         1
chi2/DOF: 0.9382; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 139; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000       13.   12561     1
Comp:  1           [NII]6583_d      28.45       17.   12561     1
-----------------------------------------------------------------
  PIXEL: 242
Emission lines included in gas templates:
['H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OI

In [4]:
import os
save_dir = "../halpha_results_Z"
os.makedirs(save_dir, exist_ok=True)

for data_file, results in all_results.items():
    galaxy_name = os.path.basename(data_file).replace("_data_orth.npy", "")

    centers, velocities, v_errs, dispersions, sigma_errs, EWs, EW_errs = [], [], [], [], [], [], []

    for center, df_rb, pp in results:
        # --- extract Hα kinematics ---
        sol_gas = pp.sol[1]
        err_gas = pp.error[1]
        v_ha, sigma_ha = sol_gas
        v_ha_err, sigma_ha_err = err_gas

        # Hα index in gas templates
        idx = np.where(pp.gas_names == "Halpha")[0][0]

        flux_ha   = pp.gas_flux[idx]
        continuum = pp.bestfit - pp.gas_bestfit
        peak_i    = np.argmax(pp.gas_bestfit_templates[:, idx])
        cont0     = continuum[peak_i]
        cont_err  = pp.noise[peak_i]
        flux_err  = pp.gas_flux_error[idx]

        EW     = flux_ha / cont0
        EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)

        # store
        centers.append(center)
        velocities.append(v_ha)
        v_errs.append(v_ha_err)
        dispersions.append(sigma_ha)
        sigma_errs.append(sigma_ha_err)
        EWs.append(EW)
        EW_errs.append(EW_err)

    # Build DataFrame
    df = pd.DataFrame({
        "center":        centers,
        "velocity":      velocities,
        "velocity_err":  v_errs,
        "dispersion":    dispersions,
        "dispersion_err": sigma_errs,
        "EW":            EWs,
        "EW_err":        EW_errs
    })

    # Save one CSV per galaxy
    out_path = os.path.join(save_dir, f"{galaxy_name}_halpha_results_orth.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved {out_path}")


/var/folders/r3/l94405xd0mq2s62v2swzjq2m0000gn/T/ipykernel_7523/1920598586.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)
/var/folders/r3/l94405xd0mq2s62v2swzjq2m0000gn/T/ipykernel_7523/1920598586.py:28: RuntimeWarning: invalid value encountered in scalar multiply
  EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)


Saved ../halpha_results_Z/WB1_halpha_results_orth.csv
Saved ../halpha_results_Z/WB2_halpha_results_orth.csv
Saved ../halpha_results_Z/WB5_halpha_results_orth.csv
Saved ../halpha_results_Z/WB6_halpha_results_orth.csv
Saved ../halpha_results_Z/WB7_halpha_results_orth.csv
